# **GEO exploratory dataset analysis**

In [115]:
import pandas as pd
import os
from mygene import MyGeneInfo
from concurrent.futures import ThreadPoolExecutor
from pyensembl import EnsemblRelease
import time
import pickle

AUGMENTATION = 0
CANCER_TYPE = "brain"

# Specify the path to the parent directory containing the folders to iterate through
data_geo_file_path = f"data_geo_{CANCER_TYPE}_{AUGMENTATION}.pkl"
all_data_columns_file_path = "./all_data_columns.pkl"
encoder_file = f"./encoder_{AUGMENTATION}.pkl"
model_file_name = f"xgboost-{AUGMENTATION}.model"

# load annotation file
#annot = pd.read_csv('../data/data-geo/colon/Human.GRCh38.p13.annot.tsv', delimiter='\t', index_col = 0)

#load data from GEO
data_geo = pd.read_csv('../data/data-geo/brain/GSE119834_norm_counts_TPM_GRCh38.p13_NCBI.tsv', delimiter='\t', index_col = 0)
#print(data_geo)

# Replace first column of data_geo with second column of annot
#data_geo.index = annot.iloc[:, 0]
#print(data_geo)

data_geo = data_geo.transpose()
print("transposed")
print(data_geo)

# Select the first 1000 columns
#data_geo = data_geo.iloc[:, :1000]

print("data_geo loaded!")
print(data_geo.columns)
print(f'Total geo columns: {len(data_geo.columns)}')
print(f'Unique geo columns: {len(set(data_geo.columns))}')

# store the count matrices and labels into pickle files
#with open(data_geo_file_path, 'wb') as f:
    #pickle.dump(data_geo, f)

transposed
GeneID      100287102  653635     102466751  107985730  100302278  645520      
GSM3384758    0.09273      2.988     2.2530    0.04746     0.0000    0.02259  \
GSM3384759    1.29000      9.081    21.7000    0.76170     1.7820    0.94290   
GSM3384760    0.05769      6.088     5.1390    0.00000     0.0000    0.00000   
GSM3384761    1.79300     13.730     4.8880    0.05616     0.2190    0.00000   
GSM3384762    0.17120      5.008     5.5440    0.00000     0.0000    0.08341   
...               ...        ...        ...        ...        ...        ...   
GSM3384851    0.10030      2.450     1.3930    0.04401     0.0000    0.00000   
GSM3384852    0.06520      3.973     3.5640    0.00000     0.0000    0.02383   
GSM3384853    0.08953      3.846     6.1630    0.00000     0.0000    0.00000   
GSM3384854    0.07361      2.997     0.3576    0.00000     0.0000    0.00000   
GSM3384855    0.77390     11.750     8.8000    0.10110     0.1971    0.00000   

GeneID      79501      10099

In [117]:
# verify if the all_data_file_path exist or not
if not os.path.exists(data_geo_file_path):

    # Converting gene id to gene name
    def convert_gene_id_to_name(gene_id):
        mg = MyGeneInfo()
        gene_info = mg.getgene(gene_id)

        if gene_info is not None and 'symbol' in gene_info:
            gene_name = gene_info['symbol']
        else:
            gene_name = None

        return gene_name

    # Function to process a column and return the gene name
    def process_column(col):
        gene_name = convert_gene_id_to_name(col)
        return gene_name

    # Use ThreadPoolExecutor to parallelize column processing
    with ThreadPoolExecutor() as executor:
        # Create a list of column names
        columns_list = data_geo.columns.tolist()

        # Process columns in parallel
        results = executor.map(process_column, columns_list)

        # Replace column names with gene names in the DataFrame
        for col, gene_name in zip(columns_list, results):
            data_geo.rename(columns={col: gene_name}, inplace=True)

    print(data_geo)
    print("Done. Gene ID replaced by Gene name in data_geo!")
    
    # store the count matrices and labels into pickle files
    with open(data_geo_file_path, 'wb') as f:
        pickle.dump(data_geo, f)

else: # .pkl file exists, load everything from it to skip processing

    with open(data_geo_file_path, 'rb') as f:
        data_geo = pickle.load(f)

ConnectionError: HTTPConnectionPool(host='mygene.info', port=80): Max retries exceeded with url: /v3/gene/101927366 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13e2ca3b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

No duplicated columns, no need to merge

In [ ]:
#open the list of genes from tcga
with open(all_data_columns_file_path, 'rb') as all_data_columns_pckl:
    all_data_columns = pickle.load(all_data_columns_pckl)

In [ ]:
print("length data_geo.columns: ", len(data_geo.columns))
print("length all_data_columns: ", len(all_data_columns))

# Delete all columns from data_geo dataframe that are not present in all_data_columns list
columns_to_drop = [col for col in data_geo.columns if col not in all_data_columns]
print("length columns_to_drop: ", len(columns_to_drop))

data_geo_filtered = data_geo.drop(columns=columns_to_drop)
print("data_geo_filtered shape: " , data_geo_filtered.shape)
      
# For each column name in all_data_columns list add a column of zeros to data_geo_filtered dataframe 
# if the column with that name does not exist, otherwise ignore it

# Create a DataFrame with zeros for missing columns
zeros_df = pd.DataFrame(0, index=data_geo_filtered.index, columns=all_data_columns)

# Concatenate data_geo_filtered and zeros_df
data_geo_filtered = pd.concat([data_geo_filtered, zeros_df], axis=1)

# Remove duplicate columns
data_geo_filtered = data_geo_filtered.loc[:, ~data_geo_filtered.columns.duplicated()]


# Construct validation dataset with proper column ordering defined in all_data_columns
data_geo_test = data_geo_filtered[all_data_columns]
print("data_geo_test shape: ", data_geo_test.shape)

To do: compute statistics about column merging and deletion process

In [ ]:
# check if there is any string in the data and print the value if exists

for i in data_geo_test:
    column_data = data_geo_test[i]
    if column_data.dtype == object and column_data.str.contains('[a-zA-Z]').any():
        string_values = column_data[column_data.astype(str).str.contains('[a-zA-Z]')]
        for string_value in string_values.unique():
            sample_indices = column_data[column_data == string_value].index
            print("Column", i, "contains the string value", string_value, "at sample indices:")
            print(sample_indices)

In [ ]:
# confirming that columns contain none zero values, as expected
print(data_geo_test.max())

# Validate data geo on trained tcga classifier

In [ ]:
# load the xgboost model
with open(model_file_name, 'rb') as f:
    model = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score

# load label encoder from pickle file
with open(encoder_file, 'rb') as f:
    label_encoder = pickle.load(f)
    
# create dataframe for storing labels

all_labels = [CANCER_TYPE] * data_geo_test.shape[0]
print(len(all_labels))

# encoding the test data with the same label encoder used in training
all_labels_encoded = label_encoder.transform(all_labels)
print(all_labels_encoded)

# sort the data_geo_test columns
data_geo_test = data_geo_test[sorted(data_geo_test.columns)]

In [ ]:
print("Verifying accuracy...")
# verify accuracy of the test data
pred_test = model.predict(data_geo_test)

accuracy = sum(pred_test == all_labels_encoded) / float(len(pred_test))

print("Accuracy of the test: %.4f%%" % (accuracy * 100.0))